In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import datetime
import os
import sys
import time

In [5]:
sys.path.append("/workspace/main-pe/")

In [6]:
import tensorflow as tf
tf.__version__

'2.0.0-alpha0'

In [8]:
from data.datasets.mpii import MpiiSinglePerson
from data.utils.data_utils import TEST_MODE, TRAIN_MODE, VALID_MODE
from data.loader import BatchLoader

In [9]:
from model import config
from model import callbacks
from model.utils import pose_format

In [18]:
from experiments.common import exp_init

In [12]:
# for custom objects
from model.activations import channel_softmax_2d
from model.losses import reconstruction_loss, pose_loss

# datasets

In [13]:
conf = {
    'pose_blocks': 2,
    'batch_size': 16,
    'dim': 2,
    'n_joints': 16,
    'dataset_name': 'mpii',
    'n_epochs': 2
}

In [14]:
mpii_path = '/workspace/datasets/mpii'
mpii = MpiiSinglePerson(mpii_path, dataconf=config.mpii_dataconf)

In [15]:
data_tr_mpii = BatchLoader(mpii,
                           ['frame'],
                           ['frame'] + ['pose'] * conf['pose_blocks'],
                           TRAIN_MODE,
                           batch_size=conf['batch_size'],
                           shuffle=False)

y_dict ['frame', 'pose', 'pose'], allkeys ['frame', 'frame', 'pose', 'pose']


In [16]:
mpii_eval = MpiiSinglePerson(mpii_path, dataconf=config.mpii_dataconf)
data_val_mpii = BatchLoader(
    mpii_eval, 
    ['frame'], 
    ['frame', 'pose', 'afmat', 'headsize'], 
    mode=VALID_MODE,
    shuffle=False)

len(data_val_mpii)

y_dict ['frame', 'pose', 'afmat', 'headsize'], allkeys ['frame', 'frame', 'pose', 'afmat', 'headsize']


124

# non nested model

In [17]:
from model.networks.old_old_mbm import MultiBranchModel
model = MultiBranchModel(dim=conf['dim'], nb_pose_blocks=conf['pose_blocks'])
model.build()

/usr/local/lib/python3.5/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 3s 0us/step
pose shape (None, 16, 2), vis shape (None, 16, 1), concat shape (None, 16, 3)
pose shape (None, 16, 2), vis shape (None, 16, 1), concat shape (None, 16, 3)
Build E_a 8.960746049880981, build E_p 4.9499852657318115, decoder D 0.9130415916442871
Input shape (None, 256, 256, 3)
Shape concat (None, 8, 8, 2048)
Outputs shape [(None, 256, 256, 3), (None, 16, 3), (None, 16, 3)]
rec y_pred shape (None, 256, 256, 3)
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
appearance_model (Model)        (None, 8, 8, 2048)   23587712    input_14[0][0]                   
____________

In [19]:
conf['exp_type'] = '2d_OLD_OLD_MBM_SAVE_TEST_NB'
model_folder = exp_init(conf, base_folder='/workspace')

Conducting experiment for 2 epochs and 2 blocks in folder /workspace/pe_experiments/exp_20190514_1744_2d_OLD_OLD_MBM_SAVE_TEST_NB__2b_bs16


In [20]:
from model import callbacks
cb_list = []
cb_list.append(callbacks.SaveModel(model_folder))
model.train(data_tr_mpii, steps_per_epoch=10, model_folder=model_folder, n_epochs=2, cb_list=cb_list)

Training with 1 callbacks
Epoch 1/2


KeyboardInterrupt: 